# Build and Deploy Many Models Leveraging Cancer Gene Expression Data With SageMaker Pipelines and SageMaker Multi-Model Endpoints

When building machine learning models that leverage genomic data, a key problem is how to allow users to select which features should be used when querying models. To address this,data scientists will sometimes build multiple models to handle specific sub-problems within the dataset. In the context of survival analysis for cancer, a common approach is to analyze gene signatures, and to predict the survival of patients based on the gene expression signatures. See [here](https://www.nature.com/articles/s41598-021-84787-5) for a an example of such an approach in the context of a number of different cancer types. See also [this](https://pubmed.ncbi.nlm.nih.gov/31296308/) review, which discusses different techniques to perform survival analysis.

A problem that may occur is that, should an application require publishing models based on many hundreds or thousands of gene signatures, managing and deploying all such models may become difficult to maintain and thus unweildly. In this blog post, we show how you can leverage SageMaker Pipelines and SageMaker MultiModel Endpoints to build and deploy many such models. 

To give a specific example, we will leverage the sample cancer RNA expression dataset discussed in the paper [Non-Small Cell Lung Cancer Radiogenomics Map Identifies Relationships between Molecular and Imaging Phenotypes with Prognostic Implications](https://pubmed.ncbi.nlm.nih.gov/28727543/). To simpify the use case, we will focus on 21 co-expressed groups that have been found in this paper to be clicially significant in NSCLC (see that paper, Table 2). These groups of genes, which the authors term metagenes, are annotated in different cellcular pathways. For example, the first group of genes LRIG1, HPGD and GDF15 are relate to the EGFR signaling pathay, while CIM,LMO2 and EFR2 all are involved in cell hypoxia/inflaation. Thus, each cancer patient (row) has gene expression values (columns). In addtion, each of the 199 patients is annoted by their survival status; each described by their Survival Status (1 for deceased; 0 for alive at time of collection of the dataset. We followed the preprocessing [this blog post](https://aws.amazon.com/blogs/industries/building-scalable-machine-learning-pipelines-for-multimodal-health-data-on-aws/) for preprocessing the data. As described more fully in that blog post, the final dataset is 119 patients where each cancer patient (row) has gene expression values (columns). If you run the pipeline described in that blog post, you will get the entire gene expression profile based on the raw FASTQ files, or you can also access the entire gene expression at [GEO](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE103584). 

The architecture for this approach is as follows:

![](images/Architecture.jpeg)

As can be seen in the diagram, we first start with data that is located in S3. We then create a [SageMaker Pipeline](https://sagemaker-examples.readthedocs.io/en/latest/sagemaker-pipelines/index.html). SageMaker Pipelines is a powerful feature that allows data scientists to wrap different components of their workload as a pipeline. This allows for a deployment strategy whereby each step of the analysis is automatically kicked off after the previous job finishes. See the associate code repository ?? for the specific syntax for creating a SageMaker Pipeline.
The pipeline consists of:

* A SageMaker Processing job for preprocessing the data

* A SageMaker Training job for training the model. 

* A SageMaker Processing job for evaluating and registering the model in SageMaker Model Registry.

* A seperate SageMaker Processing job for deploying the model on SageMaker Multi Model Endpoint (MME)




Before we begin lets verify SageMaker version

In [3]:
import sagemaker
sagemaker.__version__

'2.109.0'

In [2]:
%pip install --upgrade --quiet sagemaker==2.109.0

Keyring is skipped due to an exception: 'keyring.backends'
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytest-astropy 0.8.0 requires pytest-cov>=2.0, which is not installed.
pytest-astropy 0.8.0 requires pytest-filter-subpackage>=0.1, which is not installed.
docker-compose 1.29.2 requires PyYAML<6,>=3.10, but you have pyyaml 6.0 which is incompatible.
aiobotocore 2.4.1 requires botocore<1.27.60,>=1.27.59, but you have botocore 1.29.24 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


* Please restart the kernel after the sagemaker update. You can do that by following the options on the menu Kernel->Restart Kernel.
* After restarting execute the from below. Make sure that the version of the sagemaker is updated '>=2.94.0'.

In [3]:
import sagemaker
sagemaker.__version__

'2.109.0'

Then let's import rest of the packages needed.

In [4]:
import time
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sagemaker import get_execution_role

from sagemaker.multidatamodel import MultiDataModel

from sagemaker.pytorch import PyTorch
from sagemaker.pytorch.model import PyTorchModel

from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.workflow.fail_step import FailStep
from sagemaker.workflow.functions import Join
from sagemaker.model_metrics import MetricsSource, ModelMetrics
from sagemaker.workflow.functions import Join
from sagemaker.workflow.model_step import ModelStep
from sagemaker.workflow.conditions import ConditionLessThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import JsonGet

from sagemaker.predictor import Predictor

import matplotlib.pyplot as plt

### Read the data 

Data related to the project is available in the `data` folder. Lets read the and do some exploratory analysis of it and basic pre-processing.

In [5]:
genomic_data_with_label = pd.read_csv("data/Genomic-data-119patients.csv")
genomic_data_with_label

,Case_ID,LRIG1,HPGD,GDF15,CDH2,POSTN,VCAN,PDGFRA,VCAM1,CD44,...,CD37,VIM,LMO2,EGR2,BGN,COL4A1,COL5A1,COL5A2,SurvivalStatus,PathologicalMstage
0,R01-005,25.438505,2.434890,4.83280,0.000000,14.80001,13.816834,3.791663,3.903575,44.62658,...,1.590920,70.972300,0.00000,0.000000,6.22329,3.268430,9.394910,11.402356,1,0
1,R01-012,23.192370,8.997890,43.86000,2.576560,57.67060,22.893300,6.526380,8.768820,32.54150,...,6.934750,158.976000,3.14894,2.828400,28.85350,22.221980,12.066000,22.500950,0,0
2,R01-013,77.553001,190.529190,7.76279,0.000000,11.79248,26.137328,6.983622,3.591960,152.81587,...,5.048800,115.247000,2.26644,3.317050,5.77029,5.490720,7.749760,3.532330,0,0
3,R01-014,63.164522,21.455770,16.58980,0.000000,20.94288,7.123850,18.156200,2.076650,57.05985,...,4.651934,101.178000,1.32881,1.462162,21.01470,6.839590,2.572220,36.834306,0,0
4,R01-017,82.543949,66.726003,15.27520,1.817656,17.53631,5.260631,7.652980,2.341880,227.60731,...,2.631933,206.286100,1.39513,1.568070,21.19790,16.872651,4.875322,9.060370,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,R01-156,23.141790,8.542742,0.00000,0.684995,119.58400,25.027800,3.129100,2.322870,143.23510,...,4.717760,58.163200,2.62162,0.976505,27.42970,29.316800,35.525100,31.737300,1,0
115,R01-157,6.565445,0.829140,4.94241,0.000000,13.29370,26.306770,3.463640,4.649160,187.18417,...,1.333220,49.906300,1.87628,1.833738,15.31520,17.690250,21.050000,21.977010,0,0
116,R01-158,26.088220,21.237700,4.81202,0.000000,16.14340,10.719810,4.399728,1.937700,126.17160,...,1.286370,57.849300,1.97280,0.659567,7.73836,29.922400,7.174080,9.495350,0,0
117,R01-159,20.813240,3.629507,0.00000,0.000000,19.68492,24.987579,2.346060,9.746220,371.70458,...,22.198750,102.767831,3.39946,2.585660,8.66216,85.256870,6.234190,15.785505,0,0


You can see that for each patient (`Case_ID`) we have all gene expression levels, as well as SurvivalStatus. Note that this dataset also contains a pathological label for the patient. We will not be leveraging this column, but you can read more about the histopathology data associated with this dataset [here](https://aws.amazon.com/blogs/industries/building-scalable-machine-learning-pipelines-for-multimodal-health-data-on-aws/). Thus, we remove `Case_ID` and `PathologicalMstage`

In [6]:
genomic_data_with_label.drop(columns=["Case_ID", "PathologicalMstage"], inplace=True)

Next, we check the Class Balanceness

In [7]:
genomic_data_with_label.SurvivalStatus.value_counts().plot.bar()
plt.show()

While class `0` is a greater proportion of cases, there is sufficient number of class `1` to proceed without rebalancing the data.

Next, we will rescale the data column, by column.

In [8]:
genomic_data = genomic_data_with_label.drop(columns=["SurvivalStatus"])
labels = genomic_data_with_label["SurvivalStatus"]

scaler = MinMaxScaler()
genomic_data[genomic_data.columns] = scaler.fit_transform(genomic_data.to_numpy())
genomic_data                  

,LRIG1,HPGD,GDF15,CDH2,POSTN,VCAN,PDGFRA,VCAM1,CD44,CD48,...,LYL1,SPI1,CD37,VIM,LMO2,EGR2,BGN,COL4A1,COL5A1,COL5A2
0,0.186033,0.011379,0.058076,0.000000,0.044392,0.043852,0.031756,0.077312,0.096727,0.000000,...,0.000000,0.041144,0.071667,0.161186,0.000000,0.000000,0.036592,0.000000,0.037805,0.048075
1,0.168126,0.042051,0.527072,0.014627,0.188186,0.080102,0.058723,0.173671,0.068072,0.208182,...,0.111333,0.264893,0.312394,0.384688,0.132416,0.254566,0.260631,0.098762,0.050754,0.099377
2,0.601524,0.890415,0.093287,0.000000,0.034304,0.093057,0.063232,0.071140,0.353251,0.058441,...,0.061315,0.425206,0.227436,0.273630,0.095306,0.298546,0.032107,0.011580,0.029830,0.011697
3,0.486809,0.100271,0.199362,0.000000,0.064996,0.017122,0.173404,0.041129,0.126207,0.089822,...,0.064753,0.177943,0.209558,0.237899,0.055878,0.131600,0.183027,0.018608,0.004730,0.165631
4,0.641315,0.311836,0.183564,0.010318,0.053570,0.009681,0.069832,0.046382,0.530587,0.075721,...,0.000000,0.109945,0.118562,0.504841,0.058666,0.141132,0.184841,0.070888,0.015895,0.037249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,0.167722,0.039923,0.000000,0.003889,0.395853,0.088626,0.025222,0.046006,0.330534,0.023121,...,0.196905,0.061541,0.212524,0.128655,0.110241,0.087889,0.246535,0.135732,0.164479,0.142071
115,0.035565,0.003875,0.059394,0.000000,0.039339,0.093734,0.028521,0.092079,0.434741,0.036051,...,0.088405,0.078723,0.060058,0.107685,0.078899,0.165043,0.126602,0.075148,0.094307,0.096955
116,0.191213,0.099252,0.057827,0.000000,0.048898,0.031484,0.037752,0.038377,0.290076,0.026981,...,0.208866,0.090146,0.057948,0.127858,0.082958,0.059363,0.051591,0.138887,0.027039,0.039260
117,0.149158,0.016962,0.000000,0.000000,0.060777,0.088466,0.017501,0.193029,0.872251,0.424727,...,0.343994,0.162622,1.000000,0.241937,0.142950,0.232718,0.060737,0.427221,0.022482,0.068335


### Split the data Train/Test


In [9]:
X_train, X_val, y_train, y_val = train_test_split(genomic_data, labels, test_size = 0.2)


After spliting the data lets visually verify that the class distributions follow the same both in `train` and `validation` data.

In [10]:
y_train.value_counts().plot.bar()
plt.show()

In [11]:
y_val.value_counts().plot.bar()
plt.show()

### Save data

In [12]:
X_train.insert(0, "SurvivalStatus", y_train)
X_train.to_csv("./data/train_data.csv", index = False, header=True)

In [13]:
X_val.insert(0, "SurvivalStatus", y_val)
X_val.to_csv("./data/validation_data.csv", index = False, header=True)

### Prepare for SageMaker Training

In [14]:
role = get_execution_role()
session = sagemaker.Session()
bucket = session.default_bucket()

s3_prefix = "genome-survival-classification/data"

### Upload to S3

In [15]:
input_train = session.upload_data(
        path="./data/train_data.csv", bucket=bucket, key_prefix="{}/train".format(s3_prefix)
    )

input_val = session.upload_data(
        path="./data/validation_data.csv", bucket=bucket, key_prefix="{}/validation".format(s3_prefix)
    )

print("Train data : [{}]".format(input_train))
print("Val data : [{}]".format(input_val))

Train data : [s3://sagemaker-us-east-1-421720712360/genome-survival-classification/data/train/train_data.csv]
Val data : [s3://sagemaker-us-east-1-421720712360/genome-survival-classification/data/validation/validation_data.csv]


## Create the Multimodel Endpoint 

At this time we are creating the multi-model endpoint (one time configuration) to serve the models that are going to be delivered by the SageMaker piplines. Note that for now we are deploying a MME model that points to an empty collection of models; we will populate the collection of models later in the SageMaker Pipeline step. We also specify a custom inference.py script, which will allow users to choose which model to invoke. 


In [16]:
FRAMEWORK_VERSION = "1.12.0"

mme_model_data_location = "s3://{}/{}/mme-models-location".format(bucket, s3_prefix)

endpoint_name = "Genome-Survival-Prediction-MultiModel-Endpoint-{}".format(time.strftime("%H-%M-%S"))

model = PyTorchModel(model_data="./model/model.tar.gz", 
                     source_dir='src', 
                     entry_point='inference.py', 
                     role=role, 
                     framework_version=FRAMEWORK_VERSION,
                     py_version = "py38",
                     sagemaker_session=session)

 
mme = MultiDataModel(
    name = "Genome-Survival-Prediction-MME-Model-{}".format(time.strftime("%H-%M-%S")),
    model_data_prefix = mme_model_data_location,
    model = model,  # passing our model
    sagemaker_session=session,
)

mme_predictor = mme.deploy(
    initial_instance_count=1, 
    instance_type="ml.m5.large", 
    endpoint_name=endpoint_name
)

-----!

#### Check for current models (First time it should be empty)

In [18]:
list(mme.list_models())

[]

## Creating the pipeline 

At this point, the trained models are stored on S3, and the Multi-Model Enpoint can dynamically retrieve the needed model based on the user request. The user specifies not only the input data to run, but which specific model to use. 

Thinking back to the gene expression data, the following diagram represents an overview of the modeling process FIX:

![](images/image_2.jpg)

In this diagram, we first start with the original gene expression data (red indicates higher expression; blue lower expression), and then split that data into N seperate subsets of gene expression data. Model 1, for example, is built on genes 1,2,3; Model 2 on genes 4,5,6 etc. We then train multiple models, where each subsample of gene expression data is leveraged to predict survival. Note that each execution of the SageMaker Pipeline corresponds to building one model based on a gene signature.   

As mentioned in the introduction, we are leveraging a small data set for just 21 genes found to be signficant in predicting survival in lung cancer. However, you could do similair analysis with others groups of genes, such as those present in the [KEGG pathway database](https://www.genome.jp/kegg/pathway.html) or [Molecular Signatures Database](http://www.gsea-msigdb.org/gsea/msigdb/index.jsp)



In [19]:
pipeline_session = PipelineSession()

from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
    ParameterFloat,
)

input_train_data = ParameterString(
    name="InputTrainData",
    default_value=input_train,
)

input_validation_data = ParameterString(
    name="InputValidationData",
    default_value=input_val,
)

genome_group = ParameterString(
    name="genomeGroup",
    default_value="ALL",
)

training_instance_type = ParameterString(
    name="TrainingInstanceType", 
    default_value="ml.m5.large"
)

mme_model_location = ParameterString(
    name="MMEModelsLocation",
    default_value=mme_model_data_location,
)

from sagemaker.workflow.steps import CacheConfig

cache_config = CacheConfig(enable_caching=True, expire_after="PT1H")


#### Training Step

In [20]:
pytorch_estimator = PyTorch(
     source_dir="src",           
     entry_point="train.py",
     framework_version = "1.12.0",
     py_version = "py38",
     instance_type= training_instance_type,
     instance_count=1,
     role = role,
     hyperparameters = {
         "genome-group" : genome_group
     },
    sagemaker_session = pipeline_session
)

#pytorch_estimator.fit({"train_data" : input_train, "val_data": input_val})

In [21]:
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep

step_train = TrainingStep(
    name="Genome-Survival-Prediction-Training",
    estimator=pytorch_estimator,
    inputs={
        "train_data": TrainingInput(
            s3_data=input_train_data,
            content_type="text/csv",
        ),
         "val_data": TrainingInput(
            s3_data=input_validation_data,
            content_type="text/csv",
        )
    },
    cache_config=cache_config
)

/opt/conda/lib/python3.7/site-packages/sagemaker/workflow/steps.py:444: UserWarning: Profiling is enabled on the provided estimator. The default profiler rule includes a timestamp which will change each time the pipeline is upserted, causing cache misses. If profiling is not needed, set disable_profiler to True on the estimator.
  warnings.warn(msg)


#### Model evaluation Step

In [22]:
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.workflow.properties import PropertyFile
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep

framework_version = "0.23-1"

sklearn_processor = SKLearnProcessor(
    framework_version=framework_version,
    instance_type="ml.m5.large",
    instance_count=1,
    base_job_name="Genome-Survival-Prediction-Eval",
    role=role,
    env = {
        "genomeGroup" : genome_group
    },
    sagemaker_session = pipeline_session
)

In [23]:
evaluation_report = PropertyFile(
    name="EvaluationReport", output_name="evaluation", path="evaluation.json"
)

step_eval = ProcessingStep(
    name="Genome-Survival-Prediction-Eval",
    processor=sklearn_processor,
    inputs=[
        ProcessingInput(
            source=step_train.properties.ModelArtifacts.S3ModelArtifacts,
            destination="/opt/ml/processing/model",
        ),
        ProcessingInput(
            source=input_validation_data,
            destination="/opt/ml/processing/test",
        ),
        ProcessingInput(
            source="./src",
            destination="/opt/ml/processing/code",
        )
    ],
    outputs=[
        ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation")
    ],
    code="src/evaluation.py",
    property_files=[evaluation_report],
)

In [24]:
step_fail = FailStep(
    name="Genome-Survival-Prediction-Fail",
    error_message="Execution failed due to Obective Metric was not met",
)

#### Define a Register Model Step to Create a Model Package


In [25]:
model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri="{}/evaluation.json".format(
            step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
        ),
        content_type="application/json",
    )
)

model = PyTorchModel(
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    role=role,
    entry_point="inference.py",
    source_dir = "src",
    framework_version = "1.12.0",
    py_version = "py38",
    sagemaker_session=PipelineSession()
)

# in addition, we might also want to register a model to SageMaker Model Registry
register_model_step_args = model.register(
    content_types=["application/json"],
    response_types=["application/json"],
    inference_instances=["ml.t2.medium", "ml.m5.xlarge"],
    transform_instances=["ml.m5.xlarge"],
    model_package_group_name='Genome-Survival-Prediction-Model-Package-Group',
    approval_status = "Approved"
)

step_model_registration = ModelStep(
   name="Genome-Survival-Prediction-Model-Registration",
   step_args=register_model_step_args,
)



/opt/conda/lib/python3.7/site-packages/sagemaker/workflow/pipeline_context.py:236: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  UserWarning,


#### Define MME Deployment Step


In [42]:
sklearn_processor_for_mme_deployment = SKLearnProcessor(
    framework_version=framework_version,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    base_job_name="Genome-Survival-Prediction-Deployment",
    role=role,
    env = {
        "modelPackageArn" : step_model_registration.steps[1].properties.ModelPackageArn,
        "mmeModelLocation" : mme_model_location,
        "genomeGroup" : genome_group,
        "AWS_DEFAULT_REGION": session.boto_region_name
    }
)

step_mme_deployment = ProcessingStep(
    name="Genome-Survival-Prediction-MME-Deployment",
    processor=sklearn_processor_for_mme_deployment,
    inputs=[
        
    ],
    outputs=[
        ProcessingOutput(output_name="mme_model_location", source="/opt/ml/processing/model/mme")
    ],
    code="src/mme_deployment.py"
)

### Condition Step

In [43]:
cond_lte = ConditionLessThanOrEqualTo(
    left=JsonGet(
        step_name=step_eval.name,
        property_file=evaluation_report,
        json_path="metrics.test_accuracy.value",
    ),
    right=0.4
)

step_cond = ConditionStep(
    name="Genome-Survival-Prediction-Condition",
    conditions=[cond_lte],
    if_steps=[step_fail],
    else_steps=[step_model_registration, step_mme_deployment],
)

### Create the pipeline using all the steps defined above

In [44]:
from sagemaker.workflow.pipeline import Pipeline

pipeline_name = f"Genome-Survival-Prediction-Pipeline"
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        input_train_data,
        input_validation_data,
        training_instance_type,
        genome_group,
        mme_model_location
    ],
    steps=[step_train, step_eval, step_cond]
)

In [45]:
import json

definition = json.loads(pipeline.definition())
definition

The input argument instance_type of function (sagemaker.image_uris.retrieve) is a pipeline variable (<class 'sagemaker.workflow.parameters.ParameterString'>), which is not allowed. The default_value of this Parameter object will be used to override it.


{'Version': '2020-12-01',
 'Metadata': {},
 'Parameters': [{'Name': 'InputTrainData',
   'Type': 'String',
   'DefaultValue': 's3://sagemaker-us-east-1-421720712360/genome-survival-classification/data/train/train_data.csv'},
  {'Name': 'InputValidationData',
   'Type': 'String',
   'DefaultValue': 's3://sagemaker-us-east-1-421720712360/genome-survival-classification/data/validation/validation_data.csv'},
  {'Name': 'TrainingInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.m5.large'},
  {'Name': 'genomeGroup', 'Type': 'String', 'DefaultValue': 'ALL'},
  {'Name': 'MMEModelsLocation',
   'Type': 'String',
   'DefaultValue': 's3://sagemaker-us-east-1-421720712360/genome-survival-classification/data/mme-models-location'}],
 'PipelineExperimentConfig': {'ExperimentName': {'Get': 'Execution.PipelineName'},
  'TrialName': {'Get': 'Execution.PipelineExecutionId'}},
 'Steps': [{'Name': 'Genome-Survival-Prediction-Training',
   'Type': 'Training',
   'Arguments': {'AlgorithmSpecificatio

In [46]:
pipeline.upsert(role_arn=role)

The input argument instance_type of function (sagemaker.image_uris.retrieve) is a pipeline variable (<class 'sagemaker.workflow.parameters.ParameterString'>), which is not allowed. The default_value of this Parameter object will be used to override it.


{'PipelineArn': 'arn:aws:sagemaker:us-east-1:421720712360:pipeline/genome-survival-prediction-pipeline',
 'ResponseMetadata': {'RequestId': 'd46e69e6-214a-4784-983b-888c2f5a88e3',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd46e69e6-214a-4784-983b-888c2f5a88e3',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '103',
   'date': 'Fri, 16 Dec 2022 15:30:24 GMT'},
  'RetryAttempts': 0}}

If you are using SageMaker Studio, you can visualize what each step of the pipeline actually looks like:

![](images/image_3.jpg)

### Start the pipeline with all the Gene groups.

In [47]:
execution = pipeline.start({
        "genomeGroup" : "ALL"
    }
)

### Pipeline Operations: Examining and Waiting for Pipeline Execution

Describe the pipeline execution

In [48]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:421720712360:pipeline/genome-survival-prediction-pipeline',
 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:421720712360:pipeline/genome-survival-prediction-pipeline/execution/lxs3vcvk7d4r',
 'PipelineExecutionDisplayName': 'execution-1671204625908',
 'PipelineExecutionStatus': 'Executing',
 'PipelineExperimentConfig': {'ExperimentName': 'genome-survival-prediction-pipeline',
  'TrialName': 'lxs3vcvk7d4r'},
 'CreationTime': datetime.datetime(2022, 12, 16, 15, 30, 25, 824000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2022, 12, 16, 15, 30, 25, 824000, tzinfo=tzlocal()),
 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:421720712360:user-profile/d-6dy9c2r2izfc/sagemakeruser',
  'UserProfileName': 'sagemakeruser',
  'DomainId': 'd-6dy9c2r2izfc'},
 'LastModifiedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:421720712360:user-profile/d-6dy9c2r2izfc/sagemakeruser',
  'UserProfileName': 'sagemakeruser',
  'Dom

Wait for the execution to complete.


In [49]:
execution.wait()

### Verify how many models deploye on MME

In [50]:
list(mme.list_models())

['/model-ALL.tar.gz']

* We can see there is model suffixed with 'ALL' already in the MME location. Let's do some predictions with the test dataset. 

### Predict with trained models using test data


In [51]:
predictor = Predictor(endpoint_name = endpoint_name)

predictor.serializer = sagemaker.serializers.JSONSerializer()
predictor.deserializer = sagemaker.deserializers.CSVDeserializer()

In [52]:
payload = {
    "inputs" : X_val.iloc[:, 1:].values
}

predictor.predict(payload, target_model="/model-ALL.tar.gz")

[['0'],
 ['1'],
 ['1'],
 ['0'],
 ['0'],
 ['0'],
 ['1'],
 ['1'],
 ['0'],
 ['1'],
 ['0'],
 ['1'],
 ['0'],
 ['1'],
 ['0'],
 ['0'],
 ['0'],
 ['1'],
 ['0'],
 ['0'],
 ['1'],
 ['0'],
 ['0'],
 ['0']]

### Next lets start training model with the "metagene_19" Gene group

In [53]:
execution = pipeline.start(
    parameters=dict(
        genomeGroup="metagene_19"
    )
)

In [54]:
execution.wait()

### Verify how many models deploye on MME

In [55]:
list(mme.list_models())

['/model-ALL.tar.gz', '/model-metagene_19.tar.gz']

We can see there is a new model suffixed with 'metagene_19' in the MME location. Let's do some predictions with the test dataset. 

In [56]:
payload = {
    "inputs" : X_val[['LRIG1', 'HPGD', 'GDF15']].iloc[0:5, :].values
}
payload

{'inputs': array([[0.07959922, 0.03087836, 0.19559475],
        [0.05884028, 0.0156981 , 0.30814739],
        [0.        , 0.00898249, 0.05668478],
        [0.13264582, 0.05260354, 0.08079599],
        [0.10139428, 0.24103913, 0.08870376]])}

In [57]:
predictor.predict(payload, target_model="/model-metagene_19.tar.gz")

[['0'], ['1'], ['0'], ['0'], ['0']]

## Clean up

Once you are completed the work with the notebook, please delete the endpoint by uncommenting the following code.

In [58]:
#predictor.delete_endpoint()